# Encoder

> Architectures and functions for creating encoders that create the embeddings

In [31]:
#export
import pandas as pd
import numpy as np
from fastcore.all import *
from tsai.callback.MVP import *
from tsai.imports import *
from tsai.models.InceptionTimePlus import InceptionTimePlus
from tsai.models.explainability import get_acts_and_grads
from tsai.models.layers import *
from tsai.data.validation import combine_split_data

In [32]:
#hide
from tsai.all import *

### Architectures

In [33]:
#export 
class DCAE_torch(Module):
    def __init__(self, c_in, seq_len, delta, nfs=[64, 32, 12], kss=[10, 5, 5],
                 pool_szs=[2,2,3], output_fsz=10):
        """
        Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions,
        sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be
        contained in the Dense layer of the network. The the number of features
        maps (filters), the filter size and the pool size can also be adjusted."
        """
        assert all_equal([len(x) for x in [nfs, kss, pool_szs]], np.repeat(len(nfs), 3)), \
            'nfs, kss, and pool_szs must have the same length'
        assert np.prod(pool_szs) == nfs[-1], \
            'The number of filters in the last conv layer must be equal to the product of pool sizes'
        assert seq_len % np.prod(pool_szs) == 0, \
            'The product of pool sizes must be a divisor of the window size'
        layers = []
        for i in range_of(kss):
            layers += [Conv1d(ni=nfs[i-1] if i>0 else c_in, nf=nfs[i], ks=kss[i]),
                       nn.MaxPool1d(kernel_size=pool_szs[i])]
        self.downsample = nn.Sequential(*layers)
        self.bottleneck = nn.Sequential(OrderedDict([
            ('flatten', nn.Flatten()),
            ('latent_in', nn.Linear(seq_len, delta)),
            ('latent_out', nn.Linear(delta, seq_len)),
            ('reshape', Reshape(nfs[-1], seq_len // np.prod(pool_szs)))
        ]))
        layers = []
        for i in reversed(range_of(kss)):
            layers += [Conv1d(ni=nfs[i+1] if i != (len(nfs)-1) else nfs[-1],
                              nf=nfs[i], ks=kss[i]),
                       nn.Upsample(scale_factor=pool_szs[i])]
        layers += [Conv1d(ni=nfs[0], nf=c_in, kernel_size=output_fsz)]
        self.upsample = nn.Sequential(*layers)


    def forward(self, x):
        print(x)
        x = self.downsample(x)
        x = self.bottleneck(x)
        x = self.upsample(x)
        return x

In [34]:
#export 
from nbs.orelm.utils import *
from nbs.orelm.foselm_torch import *
from nbs.orelm.linear_recurrent import *
import nbs.orelm.elm_torch as elm
import nbs.orelm.mse_loss_i as mse
class ORELM_torch(elm.ELM_torch):
    def valid_parameters(self, inputs, outputs, numHiddenNeurons, activationFunction, LN,AE, ORTH, inputWeightForgettingFactor, outputWeightForgettingFactor, seq_len):
        assert isinstance(inputs, (int, float)) and inputs >= 1, \
          'inputs must be numeric and greater than or equal to 1'
        assert isinstance(outputs, (int, float)) and outputs >= 1, \
          'outputs must be numeric and greater than or equal to 1'
        assert isinstance(numHiddenNeurons, (int, float)) and numHiddenNeurons >= 1, \
          'numHiddenNeurons must be numeric and greater than or equal to 1'
        assert isinstance(ORTH, bool), \
          'orth must be a logical value'
        assert isinstance(inputWeightForgettingFactor, (int, float)) and 0 < inputWeightForgettingFactor <= 1, \
          'inputWeightForgettingFactor must be numeric between 0 and 1'
        assert isinstance(outputWeightForgettingFactor, (int, float)) and 0 < outputWeightForgettingFactor <= 1, \
          'outputWeightForgettingFactor must be numeric between 0 and 1'
        assert seq_len > 0, \
            'Sequence len (seq_len) must be bigger than 0'

    def __init__(
        self, 
        inputs, 
        outputs, 
        numHiddenNeurons, 
        activationFunction = "sig", #? Must always be "sig"
        LN    = True,  #? - Layer normalization boolean
        AE    = True,  #? - Para Alaiñe, siempre es True
        ORTH  = True, 
        inputWeightForgettingFactor   = 0.999,
        outputWeightForgettingFactor  = 0.999,
        seq_len = 1,
    ): 
        super().__init__(inputs, outputs, numHiddenNeurons)
        self.valid_parameters(inputs, outputs, numHiddenNeurons, activationFunction, LN,AE, ORTH, inputWeightForgettingFactor, outputWeightForgettingFactor, seq_len)
        print("inputs: " + str(inputs))
        print("outputs: " + str(outputs))
        print("numNeurons: " + str(numHiddenNeurons))
        print("Out weight FF: " + str(outputWeightForgettingFactor))
        print("Window size: " + str(seq_len))
        
        self.initialized = 0
        
        self.activationFunction = activationFunction #?
        self.outputs            = outputs
        self.numHiddenNeurons   = numHiddenNeurons
        self.inputs             = inputs
        self.window_size        = seq_len

        # input to hidden weights
        print("("+str(self.numHiddenNeurons) +", "+ str(self.inputs)+")")
        self.get_random_InputWeights()
        # hidden layer to hidden layer wieghts
        self.get_random_HiddenWeights()
    
        # initial hidden layer activation
        self.initial_H  = self.get_random_matrix(1, self.numHiddenNeurons) * 2 - 1
        self.H          = self.initial_H
        
        print("orelm-H ~ ", self.H.shape)
        self.LN         = LN #?
        self.AE         = AE #? 
        self.ORTH       = ORTH
        
        # bias of hidden units
        self.get_random_Bias()
        # hidden to output layer connection
        self.beta = self.get_random_matrix(self.numHiddenNeurons, self.outputs)
        
        # auxiliary matrix used for sequential learning
        self.M = torch.inverse(0.00001 * torch.eye(self.numHiddenNeurons)) #eye: diagonal, inv: inverse
        
        self.forgettingFactor       = outputWeightForgettingFactor
        self.inputForgettingFactor  = inputWeightForgettingFactor

        self.trace      = 0
        self.thresReset = 0.001
        print("-------- num_inputs, window_size = " + str(self.inputs) + "," + str(self.window_size))

        if self.AE: #En OTSAD -> directamente FOSELM
            self.inputAE = FOSELM_torch(
                inputs              = inputs,
                outputs             = inputs,
                numHiddenNeurons    = numHiddenNeurons,
                activationFunction  = activationFunction, #?
                LN                  = LN, #?
                forgettingFactor    = inputWeightForgettingFactor,
                ORTH = ORTH,
                seq_len = self.window_size
            )

            print("input FOSELM model", self.inputAE)
            print("input, output" ,self.inputAE.inputs, self.inputAE.outputs)

            
            self.hiddenAE = FOSELM_torch(
                inputs              = numHiddenNeurons,
                outputs             = numHiddenNeurons,
                numHiddenNeurons    = numHiddenNeurons,
                activationFunction  = activationFunction,#?
                LN                  = LN,
                ORTH                = ORTH,
                seq_len             = self.window_size
            )

            print("Output FOSELM model", self.hiddenAE)
            print("input, output", self.hiddenAE.inputs, self.hiddenAE.outputs)
        
    def __calculateInputWeightsUsingAE(self, features):
        print("--> Input AE")
        self.inputAE.train_func(features=features,targets=features)
        print("Input AE -->")
        return self.inputAE.beta

    def __calculateHiddenWeightsUsingAE(self, features):
        print("--> Hidden AE")
        self.hiddenAE.train_func(features=features,targets=features)
        print("Hidden AE -->")
        return self.hiddenAE.beta
    
    def calculateHiddenLayerActivation(self, features, flag_debug=0):
        """
        Calculate activation level of the hidden layer
        :param features feature matrix with dimension (numSamples, numInputs) #Aqui se añade numWindows
        :return: activation level (numSamples, numHiddenNeurons)
        """
        #? Este paso lo quita Alaiñe... porque sólo está implementada la opción de "sig"
        if self.activationFunction == "sig": 
            if self.AE:
                self.inputWeights = self.__calculateInputWeightsUsingAE(features)
                print("Features ~ ", features.shape)
                num_samples = features.shape[0]
                self.H = torch.zeros(num_samples, self.numHiddenNeurons, self.numHiddenNeurons)
                for i in range(num_samples):
                    self.H[i]  = self.get_random_matrix(1, self.numHiddenNeurons) * 2 - 1
                print("orelm-H ~ ", self.H.shape)
                self.hiddenWeights = self.__calculateHiddenWeightsUsingAE(self.H)
            self.fprint("Before LR " + str(flag_debug) + ": " + str(features.shape), self.print_flag)
            #Linear recurrent RNN layer setup
            hidden_size = self.numHiddenNeurons     
            input_size  = hidden_size #self.inputs 
            numLayers   =1                  #Num Linear Recurrent layers
            batch_size = features.shape[0]  #numSamples
            #Create layer
            self.fprint("Create layer", self.print_flag)
            lr_layer  = nn.RNN(input_size,hidden_size,numLayers,bias=True, batch_first=True)
            #Setup bias matrices
            self.fprint("Setup bias", self.print_flag)
            lr_layer.bias_ih_l0 = nn.Parameter(self.bias)
            lr_layer.bias_hh_l0 = nn.Parameter(self.bias)
            #Create tensor for initial hiddenState
            #lr_initial_hidden_state = torch.zeros(numLayers, batch_size, self.numHiddenNeurons)  
            lr_initial_hidden_state = torch.zeros(numLayers, batch_size, features.shape[2])  
            self.fprint("Apply LR layer", self.print_flag)
            self.fprint("features ~ " + str(features.shape), self.print_flag)
            self.fprint("Lr initial hidden state ~ " + str(lr_initial_hidden_state.shape), self.print_flag)
            lr_output, lr_new_hidden_state = lr_layer(features, lr_initial_hidden_state)
            #lr_layer = linear_recurrent(features    = features,inputW      = self.inputWeights,hiddenW     = self.hiddenWeights, hiddenA     = self.H, bias        = self.bias)
            #Layer normalization
            if self.LN: #? -> Aqui es siempre true para Alaiñe
                #Batch normalization
                self.fprint("ORELM: Create normalization layer", self.print_flag)
                ln_layer  = self.get_ln_layer(lr_output)
                self.fprint("ORELM: Normalize lr output", self.print_flag)
                lr_output = ln_layer(lr_output)
            #Layer activation
            self.fprint("Get hidden layer activation", self.print_flag)
            self.H = torch.sigmoid(lr_output)
            print("orelm-H ~ ", self.H.shape)
        else:
            print ("Unknown activation function type: " + self.activationFunction )
            raise NotImplementedError
        print("ORELM: calculate hidden layer activation "+str(flag_debug)+" -->")
        return self.H
    
    def initializePhase(self, lamb=0.0001):
        """
        Step 1: Initialization phase
        :param features feature matrix with dimension (numSamples, numInputs)
        :param targets target matrix with dimension (numSamples, numOutputs)
        """

        if self.activationFunction == "sig":
            self.get_random_Bias()
        else:
            print (" Unknown activation function type: " + self.activationFunction)
            raise NotImplementedError   
    
        self.M      = torch.inverse(lamb*torch.eye(self.numHiddenNeurons))
        self.beta   = torch.zeros([self.numHiddenNeurons,self.outputs])
        
        # randomly initialize the input->hidden connections
        self.get_random_InputWeights()
        self.inputWeights = self.inputWeights * 2 - 1
        print("--> Initialize_Phase: Input Weights initialized. Shape: "+ str(self.inputWeights.shape)) #? 
        if self.AE:
            self.inputAE.initializePhase(lamb=0.00001)
            self.hiddenAE.initializePhase(lamb=0.00001)
        else:
            # randomly initialize the input->hidden connections
            self.get_random_InputWeights()
            self.inputWeights = self.inputWeights*2-1
        # ... ? Esta parte no está en Alaiñe
        if self.ORTH: 
            if self.numHiddenNeurons > self.inputs:
                self.inputWeights = orthogonalization(self.inputWeights)
        else:
            self.inputWeights = orthogonalization(self.inputWeights.t())
            self.inputWeights = self.inputWeights.t()
        # hidden layer to hidden layer weights
        self.get_random_HiddenWeights()
        self.hiddenWeights = self.hiddenWeights *2-1
        if self.ORTH:
            self.hiddenWeights = orthogonalization(self.hiddenWeights)
        #? ...
    def reset(self):#?
        self.H = self.initial_H
    

    
    def train_func(self, features, targets,RESETTING=False):
        """
        Step 2: Sequential learning phase
        :param features feature matrix with dimension (numSamples, numInputs)
        :param targets target matrix with dimension (numSamples, numOutputs)
        """
        sys.stdout.flush()
        (numSamples, numOutputs) = targets.shape
        (numWeights, _) = features.shape
        print("ORELM:TRAIN:samples = weights: " +  str(numSamples) + " | outputs: " + str(numOutputs))
        print("ORELM:TRAIN:Features shape: " + str(features.shape) + "=> Columns: " + str(features.shape[0]))
        print("ORELM:TRAIN:Weights number: " + str(numWeights) + " = " + str(numSamples))
        sys.stdout.flush()
        assert (features.shape[0] == numSamples), \
            "Number of columns of features and weights differ"
        self.fprint("--> Calculate Hidden Activation 1", self.print_flag)
        H = self.calculateHiddenLayerActivation(features, 1)
        self.fprint("Calculate Hidden Activation 1 -->", self.print_flag)
        Ht = H.t()
        try:
            scale = 1/(self.forgettingFactor)
            aux = scale * self.M
            self.M = aux -  torch.mm(aux,
                                torch.mm(Ht, torch.mm(
                                    torch.pinverse(torch.eye(numSamples) + torch.mm(H, torch.mm(aux, Ht))),
                                    torch.mm(H, aux)
                                    )
                                )
                            )
            #...? Falta en el trozo de Alaiñe
            if RESETTING:
                beforeTrace=self.trace
                self.trace=self.M.trace()
                print (torch.abs(beforeTrace - self.trace))
                if torch.abs(beforeTrace - self.trace) < self.thresReset:
                    print (self.M)
                    #eig,_=torch.eig(self.M, eigenvectors=True)
                    eig,_=torch.eig(self.M, eigenvectors=False)
                    lambMin=min(eig)
                    lambMax=max(eig)
                    #lamb = (lambMax+lambMin)/2
                    lamb = lambMax
                    lamb = lamb.real
                    self.M= lamb*torch.eye(self.numHiddenNeurons)
                    print ("reset")
                    print (self.M)
            #? ...
            aux = self.forgettingFactor*self.beta
            self.beta = aux +   torch.mm(
                                    self.M, 
                                    torch.mm(Ht, targets - torch.mm(H, aux))
                                )
        except torch.linalg.LinAlgError:
            print ("SVD not converge, ignore the current training cycle")

        print("ORELM train -->") 
    
    def orelm_splitter(self, model): #Ver cómo escribirlo bien para que no de guerra
        return [list(self.inputAE.parameters()), list(self.hiddenAE.parameters())]
    
    
    def forward(self, features): #Predict
        """
        Make prediction with feature matrix
        :param features: feature matrix with dimension (numSamples, numInputs, time_steps)
        :return: predictions with dimension (numSamples, numOutputs, time_steps)
        """

        self.fprint("--> ORELM Forward", self.print_flag)

        (_, num_inputs, _)  = features.shape 
        assert num_inputs == self.inputs, \
            "FOSELM ~ Invalid number of inputs for the model features ~ " + str(features.shape) + \
                  "num_inputs " + str( num_inputs) 

        self.fprint("--> ORELM Calculate Hidden Activation 3", self.print_flag)
        self.fprint("ORELM Features ~ " + str(features.shape), self.print_flag)
        H = self.calculateHiddenLayerActivation(features,2 )
        self.fprint("ORELM Get prediction", self.print_flag)
        prediction = torch.matmul(H, self.beta)
        self.fprint("ORELM Calculate Hidden Activation 3 --> Features ~ " + str(features.shape), self.print_flag)
        print("ORELM Forward --> result ~ " + str(prediction.shape))
        return prediction        

In [35]:
foo = torch.rand(3, 1, 48)
m = DCAE_torch(c_in=foo.shape[1], seq_len=foo.shape[2], delta=12)
m(foo).shape

tensor([[[0.7610, 0.8849, 0.3878, 0.9022, 0.6658, 0.5451, 0.7694, 0.9382,
          0.0163, 0.0407, 0.5795, 0.1361, 0.2772, 0.5478, 0.4534, 0.7783,
          0.6110, 0.5836, 0.8994, 0.0716, 0.3055, 0.3723, 0.6483, 0.7274,
          0.6773, 0.4872, 0.7221, 0.0855, 0.1757, 0.6080, 0.6779, 0.0587,
          0.0946, 0.1294, 0.2324, 0.3407, 0.1457, 0.6236, 0.9381, 0.6164,
          0.5394, 0.1681, 0.9705, 0.0111, 0.7146, 0.8209, 0.5371, 0.9354]],

        [[0.3764, 0.2050, 0.6222, 0.9329, 0.8208, 0.6351, 0.8877, 0.1968,
          0.5386, 0.3968, 0.0889, 0.9493, 0.3365, 0.7428, 0.5363, 0.8361,
          0.4192, 0.7279, 0.9594, 0.1738, 0.8017, 0.8979, 0.6681, 0.3260,
          0.9741, 0.4791, 0.0251, 0.3857, 0.7384, 0.7745, 0.6473, 0.0871,
          0.3717, 0.9305, 0.7989, 0.0722, 0.1155, 0.5438, 0.7354, 0.2599,
          0.7566, 0.0797, 0.8565, 0.1156, 0.9854, 0.1915, 0.1503, 0.4660]],

        [[0.5143, 0.7183, 0.0100, 0.0275, 0.6500, 0.1778, 0.6169, 0.5700,
          0.1096, 0.6638, 0.8608

torch.Size([3, 1, 48])

### Dictionary to get the default backbone modules to get the embeddings from

In [36]:
#export
ENCODER_EMBS_MODULE_NAME = {
    InceptionTimePlus: 'backbone', # for mvp based models
    DCAE_torch: 'bottleneck.latent_in',
    ORELM_torch: 'outputs'
}

### Getting the embeddings (activations) from the encoder

In [37]:
#export
def get_enc_embs(X, enc_learn, module=None, cpu=False, average_seq_dim=True, to_numpy=True):
    """
        Get the embeddings of X from an encoder, passed in `enc_learn as a fastai
        learner. By default, the embeddings are obtained from the last layer
        before the model head, although any layer can be passed to `model`.
        Input
        - `cpu`: Whether to do the model inference in cpu of gpu (GPU recommended)
        - `average_seq_dim`: Whether to aggregate the embeddings in the sequence dimensions
        - `to_numpy`: Whether to return the result as a numpy array (if false returns a tensor)
    """
    if cpu:
        enc_learn.dls.cpu()
        enc_learn.cpu()
    else:
        enc_learn.dls.cuda()
        enc_learn.cuda()
    if enc_learn.dls.bs == 0: enc_learn.dls.bs = 64
    aux_dl = enc_learn.dls.valid.new_dl(X=X)
    aux_dl.bs = enc_learn.dls.bs if enc_learn.dls.bs>0 else 64
    module = nested_attr(enc_learn.model,
                         ENCODER_EMBS_MODULE_NAME[type(enc_learn.model)]) \
                if module is None else module
    embs = [get_acts_and_grads(model=enc_learn.model,
                               modules=module,
                               x=xb[0], cpu=cpu)[0] for xb in aux_dl]
    embs = to_concat(embs)
    if embs.ndim == 3 and average_seq_dim: embs = embs.mean(axis=2)
    if to_numpy: embs = embs.numpy() if cpu else embs.cpu().numpy()
    return embs

In [38]:
import wandb
from dvats.utils import *
wandb_api = wandb.Api()
#enc_artifact = wandb_api.artifact('tchub/learner-mvp:run-tchub-3tipekxw')
enc_artifact = wandb_api.artifact('mi-santamaria/test-project/dcae:v0')
enc_learner = enc_artifact.to_obj()
X = torch.rand(9, 1, 48)

wandb:   1 of 1 files downloaded.  


In [39]:
#slow
%%time
embs = get_enc_embs(X, enc_learner, cpu=True)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__

UsageError: Line magic function `%%time` not found.


In [40]:
%%time
embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=False)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__, embs.device

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
%%time
embs = get_enc_embs(X, enc_learner, cpu=False, to_numpy=True)
test_eq(embs.shape[0], X.shape[0])
embs.shape, embs.__class__

## Export -

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
beep(1)